# Assignment 1 - Code Example - Part B

This achieves an accuracy of 93.26% on test set.

In [ ]:
# provide some basic operators like matrix multiplication
import numpy as np

## Some Useful Classes

In [22]:
# base class
class Module:
    def __init__(self):
        self.params = []  # 实例变量，用于存储参数

# sequential module    
class Sequential(Module):
    def __init__(self, *layers):
        super().__init__()
        self.layers = layers
        for layer in layers:
            self.params.extend(layer.params)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, dout):
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

## Softplus

This implements the [Softplus](https://pytorch.org/docs/stable/generated/torch.nn.Softplus.html) function.

$y = \frac{1}{\beta} \ln(1+e^{\beta x})$

$y' = \frac{1}{1+e^{-\beta x}}$

Default: $\beta=1$

$e^{\beta x}$ might be too large and unstable; so we use linear function to approximate it when $\beta x$ is above the threshold $20$.

In [23]:
class Softplus(Module):
    def __init__(self, beta=1.0, threshold=20.0):
        assert beta > 0 and threshold > 0
        self.beta = beta
        self.threshold = threshold

    def forward(self, x):
        self.beta_x = self.beta * x # save the input for backward use
        y = np.log(1 + np.exp(self.beta_x)) / self.beta
        y_relu = np.where(x > 0, x, 0)
        return np.where(x < self.threshold, y, y_relu)
    
    def backward(self, dy):
        grad = 1 / (1 + np.exp(-self.beta_x))
        grad_relu = np.where(self.beta_x > 0, 1, 0)
        return dy * np.where(self.beta_x < self.threshold, grad, grad_relu)

## LinearNoBias

This implements the [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) layer but without the bias term.

$y = x W^T$

$dy/dx = W$

$dy/dW = x$

In [24]:
class LinearNoBias(Module):
    def __init__(self, in_features, out_features):
        self.weight = (np.random.rand(out_features, in_features) * 2 - 1) / in_features ** 0.5
        self.weight_grad = np.zeros_like(self.weight)

    @property
    def params(self):
        return [dict(val=self.weight, grad=self.weight_grad)]

    def forward(self, x):
        self.x = x
        return x @ self.weight.T

    def backward(self, dy):
        self.weight_grad[:] = dy.T @ self.x
        return dy @ self.weight
    

## CrossEntropyLoss

This implements the [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) loss.


In [25]:
def onehot(x, num_classes=10):
    y = np.zeros([len(x), num_classes])
    y[np.arange(len(x)), x] = 1
    return y


class CrossEntropyLoss(Module):    
    def forward(self, x_logit, x_target):
        self.x_logit = x_logit
        self.x_target = x_target
        
        # softmax
        x_logit_sub = np.exp(x_logit - np.max(x_logit, axis=1, keepdims=True))
        x_softmax = x_logit_sub / np.sum(x_logit_sub, axis=1, keepdims=True)
        x_softmax = np.clip(x_softmax, a_min=1e-15, a_max=None) # to avoid zero values
        self.x_softmax = x_softmax # save for backward

        # loss of each item
        loss_x = -np.log(x_softmax)[np.arange(len(x_target)), x_target]

        # average
        return loss_x.mean()

    def backward(self, dy):
        return dy * (self.x_softmax - onehot(self.x_target)) / len(self.x_logit)

## Optimizer

In [26]:
class Optim:
    def __init__(self, params, lr=0.01):
        self.params = params
        self.lr = lr

    def zero_grad(self):
        for p in self.params:
            p["grad"][...] = 0.0

In [27]:
class SGD(Optim):
    def __init__(self, params, lr=0.01, momentum=0.9):
        super().__init__(params, lr) 
        self.momentum = momentum
        self.velocities = [np.zeros_like(p["val"]) for p in params]

    def step(self):
        for i, p in enumerate(self.params):
            # v = momentum * v + grad
            self.velocities[i] = self.momentum * self.velocities[i] + p["grad"]
            # param = param - lr * v
            p["val"] -= self.lr * self.velocities[i]

## Implemented Modules

### Sigmoid

In [28]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

### LeakyReLU

In [29]:
def leaky_relu(x, negative_slope=0.01):
    return np.where(x > 0, x, x * negative_slope)

### Linear

In [30]:
class Linear(Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # initialize weight and bias
        scale = np.sqrt(2.0 / in_features)
        self.weight = {'val': np.random.randn(in_features, out_features) * scale,
                       'grad': np.zeros((in_features, out_features))}
        self.params.append(self.weight)
        
        if bias:
            self.bias = {'val': np.zeros(out_features),
                         'grad': np.zeros(out_features)}
            self.params.append(self.bias)
        else:
            self.bias = None
        
        # initialize cache to store input for forward pass
        self.cache = None
    
    def forward(self, x):
        # store input for backward pass
        self.cache = x
        out = x @ self.weight['val']
        if self.bias is not None:
            out += self.bias['val']
        return out
    
    def backward(self, dout):
        # compute gradients
        x = self.cache
        self.weight['grad'] += x.T @ dout
        if self.bias is not None:
            self.bias['grad'] += dout.sum(axis=0)
        return dout @ self.weight['val'].T

### Conv2d

In [31]:
class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, bias=True):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride if isinstance(stride, tuple) else (stride, stride)
        self.bias = bias

        # he initialization scale factor
        scale = np.sqrt(2.0 / (in_channels * np.prod(self.kernel_size)))
        # initialize weight with random values and zero gradients
        self.weight = {
            'val': np.random.randn(out_channels, in_channels, *self.kernel_size) * scale,
            'grad': np.zeros((out_channels, in_channels, *self.kernel_size))
        }
        self.params.append(self.weight)

        if self.bias:
            self.bias_param = {'val': np.zeros(out_channels),
                               'grad': np.zeros(out_channels)}
            self.params.append(self.bias_param)
        else:
            self.bias_param = None

        self.cache = None

    def forward(self, x):
        # get input dimension
        N, C, H, W = x.shape
        # kernel height and width
        K_h, K_w = self.kernel_size
        # stride height and width
        S_h, S_w = self.stride
        # padding height and width
        P_h, P_w = K_h // 2, K_w // 2

        # calculate output dimension
        H_out = (H + 2 * P_h - K_h) // S_h + 1
        W_out = (W + 2 * P_w - K_w) // S_w + 1

        # pad input with zeros
        x_pad = np.pad(x, [(0, 0), (0, 0), (P_h, P_h), (P_w, P_w)], mode='constant')
        
        
        # im2col
        cols = np.zeros((N, C, K_h, K_w, H_out, W_out))

        for i in range(K_h):
            for j in range(K_w):
                cols[:, :, i, j, :, :] = x_pad[:, :, i:i + S_h * H_out:S_h, j:j + S_w * W_out:S_w]

        cols = cols.transpose(0, 4, 5, 1, 2, 3).reshape(N * H_out * W_out, -1)
        w_col = self.weight['val'].reshape(self.out_channels, -1).T

        # print(f"cols shape: {cols.shape}")
        # print(f"w_col shape: {w_col.shape}")

        # convolution
        out = cols @ w_col
        if self.bias_param is not None:
            out += self.bias_param['val']

        out = out.reshape(N, H_out, W_out, self.out_channels).transpose(0, 3, 1, 2)
        # cache values needed for backward pass
        self.cache = (x, cols, w_col.shape, (H_out, W_out), (P_h, P_w), S_h, S_w)
        return out

    def backward(self, dout):
        # retrieve cached values
        x, cols, w_shape, (H_out, W_out), (P_h, P_w), S_h, S_w = self.cache
        N, C, H, W = x.shape
        K_h, K_w = self.kernel_size

        # flatten output gradient
        dout_flat = dout.transpose(0, 2, 3, 1).reshape(-1, self.out_channels)
        # print(f"dout_flat shape: {dout_flat.shape}")
        
        # calculate weight gradient
        dw = cols.T @ dout_flat
        self.weight['grad'] += dw.T.reshape(self.weight['val'].shape)

        if self.bias_param is not None:
            self.bias_param['grad'] += dout_flat.sum(axis=0)

        w_flat = self.weight['val'].reshape(self.out_channels, -1)
        # print(f"w_flat.T shape: {w_flat.T.shape}")

        # check whether dimension matches
        if dout_flat.shape[1] != w_flat.shape[0]:
            raise ValueError(f"Dimension mismatch: dout_flat shape {dout_flat.shape}, w_flat shape {w_flat.shape}")

        dcols = dout_flat @ w_flat
        dcols = dcols.reshape(N, H_out, W_out, C, K_h, K_w).transpose(0, 3, 4, 5, 1, 2)

        # padded input gradient
        dx_pad = np.zeros((N, C, H + 2 * P_h, W + 2 * P_w))
        for i in range(K_h):
            for j in range(K_w):
                dx_pad[:, :, i:i + H_out * S_h:S_h, j:j + W_out * S_w:S_w] += dcols[:, :, i, j, :, :]

        dx = dx_pad[:, :, P_h:H + P_h, P_w:W + P_w]
        return dx

### Dropout

In [32]:
class Dropout:
    def __init__(self, p=0.5):
        # Dropout probability (probability of dropping a neuron)
        self.p = p
        # Mask to store which neurons are dropped
        self.mask = None
    
    def forward(self, x, training=True):
        # Apply dropout only during training
        if training:
            # Generate binary mask with probability (1 - p)
            # Scale the mask by (1 - p) to maintain the expected value
            self.mask = np.random.binomial(1, 1 - self.p, size=x.shape) / (1 - self.p)
            # Apply mask to input
            return x * self.mask

        # During inference, return the input as is
        return x

### BatchNorm2d

In [33]:
class BatchNorm2d(Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super().__init__()
        # Small constant for numerical stability
        self.eps = eps
        # Momentum for running mean and variance calculation
        self.momentum = momentum
        
        # Learnable scale parameter (gamma) initialized to ones
        self.gamma = {'val': np.ones(num_features), 'grad': np.zeros(num_features)}
        # Learnable shift parameter (beta) initialized to zeros
        self.beta = {'val': np.zeros(num_features), 'grad': np.zeros(num_features)}
        # Add gamma and beta to trainable parameters
        self.params.extend([self.gamma, self.beta])
        
        # Running mean and variance for inference
        self.running_mean = np.zeros(num_features)
        self.running_var = np.ones(num_features)
        # Cache for storing intermediate values during forward pass
        self.cache = None

    def forward(self, x, training=True):
        if training:
            # Calculate mean and variance across batch and spatial dimensions
            mu = x.mean(axis=(0,2,3), keepdims=True)
            var = x.var(axis=(0,2,3), keepdims=True)
            # Update running mean and variance with momentum
            self.running_mean = (1 - self.momentum)*self.running_mean + self.momentum*mu.squeeze()
            self.running_var = (1 - self.momentum)*self.running_var + self.momentum*var.squeeze()
        else:
            # Use running mean and variance during inference
            mu = self.running_mean.reshape(1,-1,1,1)
            var = self.running_var.reshape(1,-1,1,1)
            
        # Normalize the input
        x_hat = (x - mu) / np.sqrt(var + self.eps)
        # Scale and shift the normalized values
        out = self.gamma['val'].reshape(1,-1,1,1) * x_hat + self.beta['val'].reshape(1,-1,1,1)
        # Cache values needed for backward pass
        self.cache = (x, x_hat, mu, var)
        return out
    
    def backward(self, dout):
        # Retrieve cached values from forward pass
        x, x_hat, mu, var = self.cache
        # Get input dimensions
        N, C, H, W = x.shape
        
        # Compute gradients for gamma and beta
        dgamma = (dout * x_hat).sum(axis=(0,2,3))
        dbeta = dout.sum(axis=(0,2,3))
        # Accumulate gradients
        self.gamma['grad'] += dgamma
        self.beta['grad'] += dbeta
        
        # Compute gradient of normalized input
        dx_hat = dout * self.gamma['val'].reshape(1,-1,1,1)
        # Compute gradient of variance
        dvar = (dx_hat * (x - mu) * -0.5 * (var + self.eps)**-1.5).sum(axis=(0,2,3), keepdims=True)
        # Compute gradient of mean
        dmu = (dx_hat * -1.0 / np.sqrt(var + self.eps)).sum(axis=(0,2,3), keepdims=True) + dvar * -2.0 * (x - mu).mean(axis=(0,2,3), keepdims=True)
        # Compute gradient of input
        dx = dx_hat / np.sqrt(var + self.eps) + dvar * 2.0 * (x - mu) / (N*H*W) + dmu / (N*H*W)
        return dx

### FocalLoss

In [34]:
def focal_loss(y_pred, y_true, gamma=2.0, alpha=0.25):
    # Clip predictions to prevent log(0) or log(1) which would result in NaN
    y_pred = np.clip(y_pred, 1e-7, 1.0 - 1e-7)
    
    # Calculate cross-entropy loss
    ce_loss = -y_true * np.log(y_pred) - (1.0 - y_true) * np.log(1.0 - y_pred)
    
    # Calculate probability of true class
    pt = np.where(y_true == 1, y_pred, 1.0 - y_pred)
    
    # Calculate focal loss by modulating cross-entropy loss
    focal_loss = alpha * np.power(1.0 - pt, gamma) * ce_loss
    
    # Return mean focal loss over the batch
    return np.mean(focal_loss)

### SGD

In [35]:
# class SGD:
#     def __init__(self, lr=0.01, momentum=0.0):
#         # Learning rate for parameter updates
#         self.lr = lr
#         # Momentum factor for velocity calculation
#         self.momentum = momentum
#         # Dictionary to store velocity for each parameter
#         self.v = {}

#     def step(self, params, grads):
#         # Update each parameter using SGD with momentum
#         for key in params.keys():
#             # Initialize velocity if not already done
#             if key not in self.v:
#                 self.v[key] = np.zeros_like(params[key])
#             # Update velocity using momentum and current gradient
#             self.v[key] = self.momentum * self.v[key] + self.lr * grads[key]
#             # Update parameter using the calculated velocity
#             params[key] -= self.v[key]

### Adam

In [36]:
class Adam:
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
        # Learning rate for parameter updates
        self.lr = lr
        # Exponential decay rate for the first moment estimates
        self.beta1 = beta1
        # Exponential decay rate for the second moment estimates
        self.beta2 = beta2
        # Small constant for numerical stability
        self.eps = eps
        # Dictionary to store first moment estimates (mean)
        self.m = {}
        # Dictionary to store second moment estimates (uncentered variance)
        self.v = {}
        # Time step counter
        self.t = 0
    
    def step(self, params, grads):
        # Increment time step
        self.t += 1
        # Update each parameter using Adam optimization
        for key in params.keys():
            # Initialize moment estimates if not already done
            if key not in self.m:
                self.m[key] = np.zeros_like(params[key])
                self.v[key] = np.zeros_like(params[key])
            # Update first moment estimate (mean)
            self.m[key] = self.beta1 * self.m[key] + (1 - self.beta1) * grads[key]
            # Update second moment estimate (uncentered variance)
            self.v[key] = self.beta2 * self.v[key] + (1 - self.beta2) * grads[key] ** 2
            # Compute bias-corrected first moment estimate
            m_hat = self.m[key] / (1 - self.beta1 ** self.t)
            # Compute bias-corrected second moment estimate
            v_hat = self.v[key] / (1 - self.beta2 ** self.t)
            # Update parameter using Adam update rule
            params[key] -= self.lr * m_hat / (np.sqrt(v_hat) + self.eps)

## Your Network

In [37]:
class ReLU(Module):
    def forward(self, x):
        self.mask = x > 0
        return x * self.mask
    
    def backward(self, dout):
        return dout * self.mask

## AdaptiveAvgPool2d

In [38]:
class AdaptiveAvgPool2d(Module):
    def __init__(self, output_size=(1,1)):
        super().__init__()
        # Target output size (height, width)
        self.output_size = output_size
    
    def forward(self, x):
        # Get input dimensions: batch size, channels, height, width
        N, C, H, W = x.shape
        # Target output height and width
        H_out, W_out = self.output_size
        # Calculate stride sizes based on input and output dimensions
        h_stride = H // H_out
        w_stride = W // W_out
        
        # Initialize output tensor
        out = np.zeros((N, C, H_out, W_out))
        # Perform adaptive average pooling
        for i in range(H_out):
            for j in range(W_out):
                # Calculate window boundaries
                h_start = i * h_stride
                h_end = min(h_start + h_stride, H)
                w_start = j * w_stride
                w_end = min(w_start + w_stride, W)
                # Compute mean of the window
                out[:,:,i,j] = x[:,:,h_start:h_end,w_start:w_end].mean(axis=(2,3))
        # Cache input shape for backward pass
        self.cache = x.shape
        return out

    def backward(self, dout):
        # Get output gradient dimensions
        N, C, H_out, W_out = dout.shape
        # Retrieve cached input shape
        orig_shape = self.cache
        H, W = orig_shape[2], orig_shape[3]
        # Calculate stride sizes
        h_stride = H // H_out
        w_stride = W // W_out
        
        # Initialize input gradient tensor
        dx = np.zeros(orig_shape)
        # Distribute gradients back to input
        for i in range(H_out):
            for j in range(W_out):
                # Calculate window boundaries
                h_start = i * h_stride
                h_end = min(h_start + h_stride, H)
                w_start = j * w_stride
                w_end = min(w_start + w_stride, W)
                # Distribute gradient equally across the window
                dx[:,:,h_start:h_end,w_start:w_end] += dout[:,:,i:i+1,j:j+1] / (h_stride * w_stride)
        return dx

## ResidualBlock

In [39]:
class ResidualBlock(Module):
    def __init__(self, in_channels, out_channels, downsample=False):
        super().__init__()
        # Whether to downsample the input
        self.downsample = downsample
        # Stride for convolution operations
        stride = 2 if downsample else 1

        # First convolution layer
        self.conv1 = Conv2d(in_channels, out_channels, 3, stride, bias=False)
        # First batch normalization layer
        self.bn1 = BatchNorm2d(out_channels)
        # First ReLU activation
        self.relu1 = ReLU()
        # Second convolution layer
        self.conv2 = Conv2d(out_channels, out_channels, 3, bias=False)
        # Second batch normalization layer
        self.bn2 = BatchNorm2d(out_channels)
        # Second ReLU activation
        self.relu2 = ReLU()

        # Shortcut connection
        if downsample or in_channels != out_channels:
            # If downsampling or channel mismatch, use a 1x1 convolution
            self.shortcut = Sequential(
                Conv2d(in_channels, out_channels, 1, stride, bias=False),
                BatchNorm2d(out_channels)
            )
        else:
            # Otherwise, use identity mapping
            self.shortcut = lambda x: x

        # Collect all parameters
        self.params.extend(self.conv1.params + self.bn1.params + self.conv2.params + self.bn2.params)
        if hasattr(self.shortcut, 'params'):
            self.params.extend(self.shortcut.params)

    def forward(self, x):
        # Apply shortcut connection
        if hasattr(self.shortcut, 'forward'):
            shortcut = self.shortcut.forward(x)
        else:
            shortcut = self.shortcut(x)

        # Main path
        out = self.conv1.forward(x)
        out = self.bn1.forward(out)
        out = self.relu1.forward(out)
        out = self.conv2.forward(out)
        out = self.bn2.forward(out)
        # Add shortcut to main path
        out += shortcut
        out = self.relu2.forward(out)
        return out

    def backward(self, dout):
        # Backward pass through second ReLU
        dout = self.relu2.backward(dout)
        dout_main = dout
        dout_shortcut = dout

        # Backward pass through main path
        dout = self.bn2.backward(dout_main)
        dout = self.conv2.backward(dout)
        dout = self.relu1.backward(dout)
        dout = self.bn1.backward(dout)
        dout_conv1 = self.conv1.backward(dout)

        # Backward pass through shortcut
        if hasattr(self.shortcut, 'backward'):
            dout_short = self.shortcut.backward(dout_shortcut)
        else:
            dout_short = dout_shortcut

        # Sum gradients from main path and shortcut
        return dout_conv1 + dout_short

## ResidualMNIST network architecture

In [40]:
class ResNetMNIST(Module):
    def __init__(self):
        super().__init__()
        # Number of input channels for the first layer
        self.in_channels = 64
        # Initial convolution layer
        self.conv1 = Conv2d(1, 64, 3, bias=False)
        # Batch normalization layer
        self.bn1 = BatchNorm2d(64)
        # ReLU activation
        self.relu = ReLU()
        # Three residual layers with increasing channels
        self.layer1 = self._make_layer(64, 2)
        self.layer2 = self._make_layer(128, 2, downsample=True)
        self.layer3 = self._make_layer(256, 2, downsample=True)
        # Adaptive average pooling to reduce spatial dimensions to 1x1
        self.avgpool = AdaptiveAvgPool2d((1,1))
        # Fully connected layer for classification
        self.fc = Linear(256, 10)
        
        # Collect all parameters from layers
        self.params = []
        for layer in [self.conv1, self.bn1, self.layer1, self.layer2, self.layer3, self.avgpool, self.fc]:
            self.params += layer.params
    
    def _make_layer(self, out_channels, num_blocks, downsample=False):
        # Create a layer with multiple residual blocks
        layers = []
        # First block may downsample
        layers.append(ResidualBlock(self.in_channels, out_channels, downsample))
        self.in_channels = out_channels
        # Add remaining blocks
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return Sequential(*layers)
    
    def forward(self, x):
        # Forward pass through the network
        x = self.conv1.forward(x)
        x = self.bn1.forward(x)
        x = self.relu.forward(x)
        x = self.layer1.forward(x)
        x = self.layer2.forward(x)
        x = self.layer3.forward(x)
        x = self.avgpool.forward(x)
        # Flatten the output for the fully connected layer
        x = x.reshape(x.shape[0], -1)
        x = self.fc.forward(x)
        return x
    
    def backward(self, dout):
        # Backward pass through the network
        dout = self.fc.backward(dout)
        # Reshape gradient for adaptive pooling
        dout = dout.reshape(dout.shape[0], 256, 1, 1)
        dout = self.avgpool.backward(dout)
        dout = self.layer3.backward(dout)
        dout = self.layer2.backward(dout)
        dout = self.layer1.backward(dout)
        dout = self.relu.backward(dout)
        dout = self.bn1.backward(dout)
        dout = self.conv1.backward(dout)
        return dout

In [41]:
# net = Sequential(
#     LinearNoBias(784, 512), Softplus(),
#     LinearNoBias(512, 256), Softplus(),
#     LinearNoBias(256, 128), Softplus(),
#     LinearNoBias(128, 10),
# )

net = ResNetMNIST()

loss_fn = CrossEntropyLoss()

## Training

In [42]:
# torch and torchvision provide some very handy utilities for dataset loading
from torch.utils.data import DataLoader
import torchvision.datasets as tv_datasets
import torchvision.transforms as tv_transforms

In [43]:
# some experimental setup
num_epochs = 10
batch_size = 128
num_workers = 6
print_every = 200

In [44]:
# prepare datasets
dataset, loader = {}, {}
for data_type in ("train", "test"):
    is_train = data_type=="train"
    dataset[data_type] = tv_datasets.MNIST(
        root="./data", train=is_train, download=True, 
        transform=tv_transforms.Compose([ # preprocessing pipeline for input images
            tv_transforms.ToTensor(),
            tv_transforms.Normalize((0.1307,), (0.3081,)),
    ]))
    loader[data_type] = DataLoader(
        dataset[data_type], batch_size=batch_size, shuffle=is_train, num_workers=num_workers,
    )


In [45]:
optimizer = SGD(net.params, lr=0.01, momentum=0.9)

for epoch in range(num_epochs):

    running_loss = 0.0
    for i, (img, target) in enumerate(loader["train"]):
        img, target = img.numpy(), target.numpy()
        # img = img.reshape(-1, 784)
        
        loss = loss_fn.forward(net.forward(img), target)

        net.backward(loss_fn.backward(1.0))
        optimizer.step()
        optimizer.zero_grad()

        # print statistics
        running_loss += loss.item()
        if i % print_every == print_every - 1:
            print(f"[epoch={epoch + 1:3d}, iter={i + 1:5d}] loss: {running_loss / print_every:.3f}")
            running_loss = 0.0

print("Finished Training")

[epoch=  1, iter=  200] loss: 0.472
[epoch=  1, iter=  400] loss: 0.074
[epoch=  2, iter=  200] loss: 0.045
[epoch=  2, iter=  400] loss: 0.036
[epoch=  3, iter=  200] loss: 0.024
[epoch=  3, iter=  400] loss: 0.026
[epoch=  4, iter=  200] loss: 0.016
[epoch=  4, iter=  400] loss: 0.017
[epoch=  5, iter=  200] loss: 0.009
[epoch=  5, iter=  400] loss: 0.010
[epoch=  6, iter=  200] loss: 0.006
[epoch=  6, iter=  400] loss: 0.007
[epoch=  7, iter=  200] loss: 0.004
[epoch=  7, iter=  400] loss: 0.004
[epoch=  8, iter=  200] loss: 0.002
[epoch=  8, iter=  400] loss: 0.003
[epoch=  9, iter=  200] loss: 0.002
[epoch=  9, iter=  400] loss: 0.002
[epoch= 10, iter=  200] loss: 0.001
[epoch= 10, iter=  400] loss: 0.001
Finished Training


## Evaluate

In [46]:
# for each test image
correct, total = 0, 0
for img, target in loader["test"]:
    img, target = img.numpy(), target.numpy()
    # img = img.reshape(-1, 784)

    # make prediction
    pred = net.forward(img)

    # accumulate
    total += len(target)
    correct += (np.argmax(pred, axis=1) == target).sum()

print(f"Accuracy of the network on the {total} test images: {100 * correct / total:.2f}%")

Accuracy of the network on the 10000 test images: 99.37%
